# K-Means

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
#Load Data
def loadDataSet(filename):
    dataMat = []
    fr = open(filename)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float, curLine)
        dataMat.append(fltLine)
    return dataMat

#Compting the Euclidean Distance
def distEuclide(vecA, vecB):
    return tf.sqrt(tf.reduce_sum(tf.square(vecA - vecB), 2))

#Choose Random Centroids
def initRandomCentroid(dataSet, k):
    m = tf.shape(dataSet)[0]
    randIndices = tf.random_shuffle(tf.range(0, m))
    begin = [0,]
    size = [k,]
    size[0] = k
    centroidIndices = tf.slice(randIndices, begin, size)
    initCentroids = tf.gather(dataSet, centroidIndices)
    return initCentroids

#Assign Each sample to the Nearest Centroid
def assignToNearest(dataSet, centroids, distMeans = distEuclide):
    #Choose Euclidean Distance
    expandVectors = tf.expand_dims(dataSet, 0)
    expandCentroids = tf.expand_dims(centroids, 1)
    distance = distMeans(expandVectors, expandCentroids)
    mins = argmin(distance, 0)
    return mins

def kMeans(dataSet, k, distMeans = distEuclide, createCent = initRandomCentroid):
    initRandomCentroid(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        
    